In [39]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
import time
import random

# Build a seed list of city's urls


https://www.tripadvisor.com/FindRestaurants?geo=32655&offset=0&establishmentTypes=9900&broadened=false

In [22]:
cities = pd.read_excel('Trip Advisor Seedlist US Cities.xlsx')

In [23]:
cities['geoid'] = cities['geoid'].apply(lambda x: int(x))

In [155]:
len(cities.City.unique())

460

In [24]:
cities

,City,State,geoid
0,Detroit,Michigan,42139
1,Nashville,Tennessee,55229
2,Columbus,Ohio,50226
3,Washington DC,District of Columbia,28970
4,Cleveland,Ohio,50207
...,...,...,...
486,Evanston,Wyoming,60461
487,Riverton,Wyoming,60542
488,Colorado Springs,Colorado,33364
489,Idaho Falls,Idaho,35489


# Develop the listings scraper

In [221]:
def sleep_with_random_delay(fixed_delay=3, random_range=(0.1, 1)):
    random_delay = random.uniform(*random_range)
    total_delay = fixed_delay + random_delay
    time.sleep(total_delay)

In [198]:
folder_path = ''

### Example

In [26]:
url = "https://www.tripadvisor.com/data/graphql/ids"

payload = [
    {
        "variables": {
            "limit": 30, 
            "racRequest": None,
            "route": {
                "page": "FindRestaurants",
                "params": {
                    "establishmentTypes": ["9900"],
                    "geoId": 42139, #change
                    "broadened": False,
                    "offset": "30" #change
                }
            },
            "additionalSelections": []
        },
        "extensions": {"preRegisteredQueryId": "8d8e5fae60f5d8c3"}
    },
    
       
]
headers = {
    "accept": "*/*",
    "content-type": "application/json",
    "origin": "https://www.tripadvisor.com",
    "priority": "u=1, i",
    #"referer": "https://www.tripadvisor.com/FindRestaurants?geo=32655&offset=60&establishmentTypes=9900&broadened=false",
    "sec-ch-device-memory": "8",
    "sec-ch-ua-arch": "\"arm\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-model": "\"\"",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "same-origin",
    "sec-fetch-site": "same-origin",
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

[{"data":{"response":{"__typename":"RestaurantPresentation_NarrowGeoResponse","contentTitle":"Top Restaurants in Detroit","restaurants":[],"facets":{"onlineOptions":{"name":"Online options & offers","items":[{"id":"20692","secondaryName":null,"count":2,"tag":{"categoryId":531,"localizedName":"Online Delivery","tagId":20692},"selected":null}]},"cuisines":{"name":"Cuisines","items":[{"id":"10642","secondaryName":null,"count":16,"tag":{"categoryId":231,"localizedName":"Cafe","tagId":10642},"selected":null},{"id":"9908","secondaryName":null,"count":4,"tag":{"categoryId":231,"localizedName":"American","tagId":9908},"selected":null},{"id":"10679","secondaryName":null,"count":2,"tag":{"categoryId":231,"localizedName":"Healthy","tagId":10679},"selected":null},{"id":"10700","secondaryName":null,"count":2,"tag":{"categoryId":231,"localizedName":"Soups","tagId":10700},"selected":null},{"id":"4617","secondaryName":null,"count":2,"tag":{"categoryId":231,"localizedName":"Italian","tagId":4617},"sele

In [32]:
response.json()[0]['data']['response']['restaurants']

[]

In [48]:
cities[422:423]

,City,State,geoid
422,St. George,Utah,57119


### Scraper

In [49]:
for index, row in cities[422:423].iterrows():
    city = row['City']
    state = row['State']
    geoid = row['geoid']
    offset = 0
    
    while True:
        try:
            payload = [
                {
                    "variables": {
                        "limit": 30, 
                        "racRequest": None,
                        "route": {
                            "page": "FindRestaurants",
                            "params": {
                                "establishmentTypes": ["9900"],
                                "geoId": geoid, #change
                                "broadened": False,
                                "offset": str(offset) #change
                            }
                        },
                        "additionalSelections": []
                    },
                    "extensions": {"preRegisteredQueryId": "8d8e5fae60f5d8c3"}
                }]

            response = requests.request("POST", url, json=payload, headers=headers)
            response.raise_for_status()
            sleep_with_random_delay()
            
            response_dict = response.json()
            
            restaurants = response_dict[0]['data']['response']['restaurants']
            
            if not restaurants:
                # Break the loop if no restaurants are found
                break
                
            # Save the response JSON to a file
            file_name = f'{city}, {state}_{offset}.json'
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'w') as json_file:
                json.dump(response_dict, json_file, indent=4)

            print('Done', index, city, offset)
            offset += 30

        except Exception as e:
            print(f'Exception: {e}')
            break

            


Done 422 St. George 0


# Read scraped data

In [199]:
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

In [201]:
data = pd.DataFrame(columns=['locationId', 'name', 'url', 'types', 'reviews', 'City', 'State'])

In [202]:
for file in json_files:
    with open(os.path.join(folder_path, file), 'r') as json_file:
        json_data = json.load(json_file)
        file_name = file.split('.json')[0]
        city = file_name.split(', ')[0]
        state = file_name.split(', ')[1].split('_')[0]
        
        restaurants = json_data[0]['data']['response']['restaurants']
        
        for restaurant in restaurants:
            locationId = restaurant['locationId']
            name = restaurant['name']
            url = restaurant['detailPageRoute']['url']
            reviews = restaurant['reviewSummary']['count']
            items = []
            types = restaurant['establishmentTypes']['items']
            for item in types:
                item = item['tag']['localizedName']
                items.append(item)
            
            # Append to the DataFrame
            data = data.append({
                    'name': name,
                    'locationId': locationId,
                    'url': url,
                    'types': items,
                    'reviews' : reviews,
                    'City': city,
                    'State': state
                }, ignore_index=True)


In [203]:
len(data.City.unique())

458

In [204]:
# Find the unique cities in seed_list
cities_names = cities.City.unique()

# Find the unique cities in scraped_data
data_names = data.City.unique()

# Find the cities in seed_list that are missing in scraped_data
missing_cities = np.setdiff1d(cities_names, data_names)

In [205]:
missing_cities

array(['Lewiston', 'Pierre South Dakota.html'], dtype=object)

**Remark:** 

    Lewiston, Maine - no coffee shops
    
    Pierre South Dakota - no coffee shops

In [206]:
data

,locationId,name,url,types,reviews,City,State
0,27978157,Cherbourg Bakery,/Restaurant_Review-g50226-d27978157-Reviews-Ch...,"[Coffee & Tea, Bakeries]",0,Columbus,Ohio
1,12950318,Dunkin',/Restaurant_Review-g50226-d12950318-Reviews-Du...,[Coffee & Tea],1,Columbus,Ohio
2,12483519,Waffle House,/Restaurant_Review-g50226-d12483519-Reviews-Wa...,"[Coffee & Tea, Bakeries, Dessert]",4,Columbus,Ohio
3,851617,Dunkin',/Restaurant_Review-g50226-d851617-Reviews-Dunk...,[Coffee & Tea],13,Columbus,Ohio
4,5049536,Tim Hortons,/Restaurant_Review-g50226-d5049536-Reviews-Tim...,"[Coffee & Tea, Bakeries]",1,Columbus,Ohio
...,...,...,...,...,...,...,...
11052,4427871,Starbucks,/Restaurant_Review-g46052-d4427871-Reviews-Sta...,"[Coffee & Tea, Quick Bites]",20,Concord,New Hampshire
11053,5074865,Dunkin',/Restaurant_Review-g46052-d5074865-Reviews-Dun...,"[Coffee & Tea, Bakeries]",4,Concord,New Hampshire
11054,4817060,Dunkin',/Restaurant_Review-g46052-d4817060-Reviews-Dun...,"[Coffee & Tea, Bakeries]",9,Concord,New Hampshire
11055,4810955,Dunkin',/Restaurant_Review-g46052-d4810955-Reviews-Dun...,"[Coffee & Tea, Bakeries]",2,Concord,New Hampshire


In [207]:
unique_values = set(item for sublist in data['types'] for item in sublist)
unique_values

{'Bakeries',
 'Bars & Pubs',
 'Coffee & Tea',
 'Dessert',
 'Quick Bites',
 'Restaurants',
 'Specialty Food Market'}

In [208]:
grouped_df = pd.DataFrame(data.groupby('name')['name'].count()).rename(columns={'name': 'count'})
grouped_df = grouped_df.reset_index()
grouped_df.columns = ['name', 'count']

In [210]:
grouped_df.sort_values(by='count', ascending=False).head(50)

,name,count
4537,Starbucks,2576
1789,Dunkin',1178
1797,Dutch Bros Coffee,113
1045,Caribou Coffee,104
5262,Tim Hortons,100
4967,The Coffee Bean & Tea Leaf,73
1790,Dunkin' Donuts,68
3732,Panera Bread,34
5258,Tiger Sugar,28
469,Biggby Coffee,23


In [211]:
chain_shops = list(grouped_df[grouped_df['count'] > 1]['name'])

In [212]:
# Filter rows 
data = data[~data['name'].isin(chain_shops)]

In [216]:
data = data[data['reviews']>0]

In [219]:
data.reset_index(drop=True, inplace=True)

In [217]:
len(data.City.unique())

422

In [231]:
#data.to_csv('data.csv', index=False)

In [220]:
data

,locationId,name,url,types,reviews,City,State
0,2556749,Cafe Bella Coffee,/Restaurant_Review-g47177-d2556749-Reviews-Caf...,"[Coffee & Tea, Quick Bites]",37,Rio Rancho,New Mexico
1,12361344,Coffee Central,/Restaurant_Review-g44028-d12361344-Reviews-Co...,[Coffee & Tea],2,Southaven,Mississippi
2,498652,Perugino,/Restaurant_Review-g51862-d498652-Reviews-Peru...,[Coffee & Tea],67,Eugene,Oregon
3,2343459,Vero Espresso,/Restaurant_Review-g51862-d2343459-Reviews-Ver...,"[Coffee & Tea, Quick Bites]",54,Eugene,Oregon
4,1014118,Espresso Roma,/Restaurant_Review-g51862-d1014118-Reviews-Esp...,"[Coffee & Tea, Quick Bites]",21,Eugene,Oregon
...,...,...,...,...,...,...,...
4382,27113108,Grizzly Bean Coffee House,/Restaurant_Review-g29948-d27113108-Reviews-Gr...,[Coffee & Tea],1,Youngstown,Ohio
4383,10507418,Classic Rock Coffee & Kitchen,/Restaurant_Review-g49882-d10507418-Reviews-Cl...,[Coffee & Tea],3,Mandan,North Dakota
4384,4824290,Cappuccino On Collins,/Restaurant_Review-g49882-d4824290-Reviews-Cap...,[Coffee & Tea],3,Mandan,North Dakota
4385,7359734,The Bean & Bakery,/Restaurant_Review-g46052-d7359734-Reviews-The...,"[Coffee & Tea, Quick Bites]",21,Concord,New Hampshire


# Develop shops scraper

In [224]:
folder_path = ''

### Example

In [187]:
url = "https://www.tripadvisor.com/data/graphql/ids"

payload = [
  
    
    {
        "variables": {
            "locationId": 7594515, #change
            "offset": 0,
            "limit": 15,
            "keywordVariant": "location_keywords_v2_llr_order_30_en",
            "needKeywords": True,
            "userId": "D389EB225B1104D9A692107341C20D3D",
            "filters": [
                {
                    "axis": "SORT",
                    "selections": ["mostRecent"]
                },
                {
                    "axis": "LANGUAGE",
                    "selections": ["en"]
                }
            ],
            "prefs": {
                "showMT": True,
                "sortBy": "DATE",
                "sortType": ""
            },
            "initialPrefs": {
                "showMT": True,
                "sortBy": "DATE",
                "sortType": ""
            },
            "filterCacheKey": "locationReviewsFilters_7594515", #change
            "prefsCacheKey": "locationReviewsPrefs_7594515" #change
        },
        "extensions": {"preRegisteredQueryId": "cbe12b7ea5ddf39c"}
    },
    
]

headers = {
    "accept": "*/*",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/json",
    "cookie": "TASameSite=1; TAUnique=%1%enc%3AaOaYy7VaLyHGrnrm8hGtBF8Iqr50fjk74GR2IyE67hMvwB2QVGaX4MAiq1D1yYd7Nox8JbUSTxk%3D; TASSK=enc%3AAG2kn4NbgnkJpz7gIBAvFxNTGj8jpQE7U6yP1NWumdi3AKve75DOC5%2Fcr20A%2F8MooAL%2Btex5fJWWg%2FDOxz42sjTzWCoLQpYH%2FfyUpAyeDQH5PAYVFuHXVNOcoVHc%2Flif4A%3D%3D; TART=%1%enc%3ARXtD4M8J2mDkj4k29KRxzLggMufQhBjmRz%2FQUmnFNer5xA0NlfRS0dty7jAYQSEAZQnMIP2Q3mY%3D; OptanonAlertBoxClosed=2024-09-12T01:28:04.794Z; eupubconsent-v2=CQE0XEAQE0XEAAcABBENBGFsAP_gAEPgACiQKaNV_C5ebWli8XZUIbtkaYwP55gz4kQhBhaIEewFwBOG7BgCB2EwNAV4JiACGBAAkiLBAQNlGABUCQAAAIgRiSCMYk2MgTNKJKBAiFMRO0NYCBxmmoFDWQCY5kqssxdxmBeAAAIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAAAAAAAAAAAAA_cff_9Ln_-uB_-_wU1AJMFCogBLAkICBQMIIEAAgjCAigAAAAAkBQAQAgCAJ2BAEOsJAAAAQAAgAAAABBEACAAASABCIAAACAQAAACAQABgAADAQAECAACACwAAgABANAQCAggEAwAQMQIhTAgCAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAbRlaYFo-YLntMAyQCgQAkAHQAXABsgEQAMIAnQBcgDbAIHBAAwAOgBXAEQAMIAnQCBwYAOADoALgA2QCIAGEAXIBA4QAHAB0ANkAiABhAE6ALkAgcKABgBcAMIBA4YACAMIBA4cAGAB0ARAAwgCdAIHARXIAAgDCAQOJAAwCIAGEAgcUACgA6AIgAYQBOgEDgA.f_wACHwAAAAA; OTAdditionalConsentString=1~43.46.55.61.70.83.89.93.108.117.122.124.135.143.144.147.149.159.192.196.202.211.228.230.239.259.266.286.291.311.318.320.322.323.327.367.371.385.394.397.407.415.424.430.436.445.453.486.491.494.495.522.523.540.550.559.560.568.574.576.584.587.591.737.802.803.820.821.839.864.899.904.922.931.938.979.981.985.1003.1027.1031.1040.1046.1051.1053.1067.1092.1095.1097.1099.1107.1135.1143.1149.1152.1162.1166.1186.1188.1205.1215.1226.1227.1230.1252.1268.1270.1276.1284.1290.1301.1307.1312.1345.1356.1364.1375.1403.1415.1416.1421.1423.1440.1449.1455.1495.1512.1516.1525.1540.1548.1555.1558.1570.1577.1579.1583.1584.1591.1603.1616.1638.1651.1653.1659.1667.1677.1678.1682.1697.1699.1703.1712.1716.1721.1725.1732.1745.1750.1765.1782.1786.1800.1810.1825.1827.1832.1838.1840.1842.1843.1845.1859.1866.1870.1878.1880.1889.1899.1917.1929.1942.1944.1962.1963.1964.1967.1968.1969.1978.1985.1987.2003.2008.2027.2035.2039.2047.2052.2056.2064.2068.2072.2074.2088.2090.2103.2107.2109.2115.2124.2130.2133.2135.2137.2140.2147.2150.2156.2166.2177.2183.2186.2205.2213.2216.2219.2220.2222.2225.2234.2253.2279.2282.2292.2305.2309.2312.2316.2322.2325.2328.2331.2334.2335.2336.2337.2343.2354.2357.2358.2359.2370.2376.2377.2387.2400.2403.2405.2407.2411.2414.2416.2418.2425.2440.2447.2461.2465.2468.2472.2477.2481.2484.2486.2488.2493.2498.2501.2510.2517.2526.2527.2532.2535.2542.2552.2563.2564.2567.2568.2569.2571.2572.2575.2577.2583.2584.2596.2604.2605.2608.2609.2610.2612.2614.2621.2628.2629.2633.2636.2642.2643.2645.2646.2650.2651.2652.2656.2657.2658.2660.2661.2669.2670.2677.2681.2684.2687.2690.2695.2698.2713.2714.2729.2739.2767.2768.2770.2772.2784.2787.2791.2792.2798.2801.2805.2812.2813.2816.2817.2821.2822.2827.2830.2831.2834.2838.2839.2844.2846.2849.2850.2852.2854.2860.2862.2863.2865.2867.2869.2873.2874.2875.2876.2878.2880.2881.2882.2883.2884.2886.2887.2888.2889.2891.2893.2894.2895.2897.2898.2900.2901.2908.2909.2916.2917.2918.2919.2920.2922.2923.2927.2929.2930.2931.2940.2941.2947.2949.2950.2956.2958.2961.2963.2964.2965.2966.2968.2973.2975.2979.2980.2981.2983.2985.2986.2987.2994.2995.2997.2999.3000.3002.3003.3005.3008.3009.3010.3012.3016.3017.3018.3019.3025.3028.3034.3038.3043.3048.3052.3053.3055.3058.3059.3063.3066.3068.3070.3073.3074.3075.3076.3077.3089.3090.3093.3094.3095.3097.3099.3100.3106.3109.3112.3117.3119.3126.3127.3128.3130.3135.3136.3145.3150.3151.3154.3155.3163.3167.3172.3173.3182.3183.3184.3185.3187.3188.3189.3190.3194.3196.3209.3210.3211.3214.3215.3217.3219.3222.3223.3225.3226.3227.3228.3230.3231.3234.3235.3236.3237.3238.3240.3244.3245.3250.3251.3253.3257.3260.3270.3272.3281.3288.3290.3292.3293.3296.3299.3300.3306.3307.3309.3314.3315.3316.3318.3324.3328.3330.3331.3531.3731.3831.4131.4531.4631.4731.4831.5231.6931.7235.7831.7931.8931.9731.10231.10631.10831.11031.11531.12831.13632.13731.14237.14332.15731.16831.16931.21233.23031.25731.25931.26031.26831.27731.27831.28031.28731.28831.29631.31631.32531.33631; AMZN-Token=v2FweIB2cExXMGdZTjNtOXFkczJGakFYSzh6WEVnMVJKTU15Mk1TeUJuMEg1bkFSRmIwYzhnM0FZSXdXMWt0Umd0c0lOdmxrUE9qN2ViVkdKZEJjNHBzNWdkYW9xSm9HTkNWUXNqcXNvdXFMdHozT0tiakVSaDV5ckgyK3RtanAwTUFXeGJrdgFiaXZ4GEIwOGU3Nys5SW1IdnY3MGpmZSsvdlE9Pf8=; pbjs_sharedId=383152b6-5ed0-4f0a-ac79-71e3359ac5c5; pbjs_sharedId_cst=JCzqLGQsIg%3D%3D; _gcl_au=1.1.156252779.1726104487; _ga=GA1.1.1298674465.1726104487; _lr_env_src_ats=true; idl_env_cst=JCzqLGQsIg%3D%3D; pbjs_unifiedID=%7B%22TDID_LOOKUP%22%3A%22FALSE%22%2C%22TDID_CREATED_AT%22%3A%222024-09-12T01%3A28%3A15%22%7D; pbjs_unifiedID_cst=JCzqLGQsIg%3D%3D; ServerPool=A; TATravelInfo=V2*A.2*MG.-1*HP.2*FL.3*RS.1; TAAUTHEAT=hE851IGiMfhvEIMFABQCS_DXENAPSyOjZ55LXtF4ZEAPDj2nBT5B4RZj7Yp-SLOWWlEcqiFeDHELFysFwwJ8JARxXYRrwEuo1Crik0opW3l2QdWudVx1dAOvaFhWcguRAl3VHGsDFvbLkR5C82VOZeaZ6tH1XA8wOJiPtXBcO_rY_En6_wfDlhqa-TlhK3S5nxox_PYTRdob5jdHdY_rZdYQ1L6826_I82vl; TADCID=uXfF_MTQGtPX7lH3ABQCrj-Ib21-TgWwDB4AzTFpg4A4DlugiZHaky2GOQNIVc5OymVj2fSCZAG-X8fhnG1J5pmcGate4NL9RYw; _lr_sampling_rate=100; PMC=V2*MS.31*MD.20240911*LD.20240913; TASID=FAED50BB41FC3D26C3A1EF23F8A2045D; PAC=AEIyD_yJA1AOJLXyGDWUOeXqX6cnkW1lYJ7Aisi8T9mNNsZwlkpKtqsHmzHnlttCaWFyWtG4sKWusIO9G3eOu6hsdP1Bawdi2df_igHETN9ZO8TuU1EhQCKghZ85Yku7YqsF5dy7UY7CXOKRGS6lpjao4VzHxxBE249OTlGVf9-DJZY49jNqtfIdgum1_St0cPLrk7iGnDn8JCfKq62TjMVafWkFVJjy4hhENDdqjHtw; SRT=TART_SYNC; TATrkConsent=eyJvdXQiOiJTT0NJQUxfTUVESUEiLCJpbiI6IkFEVixBTkEsRlVOQ1RJT05BTCJ9; idl_env=AjmocZ71GjxGqRYwT0JgzR62EukfsZHfUf2BFpnT3HbNw-ISWikeLdQGLWzQbD7Dwl-ghGWSTwKagZ-LDWqbn7cy07a6FhozkRZEYI1vxSZDoPfRxE82FZ9bTXCie3W9k5pCGC-AQ0_wO-mEtxrvCw-MzuvrGuYbQAlNpJM5_B-S-Ku8vA3qQnJ9mTU8xjRZENxzSIxgjmic6epXlwpEx6mC69JCQVM-xhwRkjXlswr3FQLexf_tUi5XcdPWz6iekUhqtPeapSTKVVOpuvb0w-TMn520BOD08KB3G6xMJMoL2BwYqdPD1pMNZPI0FQJ2EjK6sDwZJmSBEd52o4wfKXsZZjdf3oziRLCkzQu4OJTCSPFQGLX6XEa4kMQQP_VpO-SLpRuTDDMk2zUKFRCe9i086QahLabcdRGEvdNcO-hi-r92d2VGRUNLZLmArD9cyBSkN8XsUaLrOULQVey1in26YGyFdxe6kbilDUCrTrM-o_NLT6MnuZFey6sxxHJKWALsR-uUFZJLuRzN6WJyLR8LVW9Agknf_OnYqZgygWoYUg0ei2EP6-WSfpsY1UDMMo_tGql1yA9VpzNvs5RwPK8QI2K5vIzgpR-4DbIVjX2i0iNPUZOM_wBU9HrLBVCOrXmGSBRZanY2ez37fyfmsrYx59LHb60Oe_FkKlWaIEmpY-HFXJA6VWe-TIbeNEbB8vo28kmndkZtqthCJNCw-Y5QmRcZ1KS2xQS2XftcdW2WOV0TDhbpwE3403uOzvcHtBGJ8KO0LWf0-r24pe5XqZzXtXAbuWp4HisGeXqqrICRMDNFZlzEfPQuu4oio_bh8gyZPHj-6bdSv5ZjP93hXQluHyxDqZ-t7__-zRHtQsmmlx5dbMjwVKGH-fzbMChd5_BJPlLAk6NGe2DdSaj5FAD9gJehuS3SXoOmEQ6G-lv2CBAguhvy9Bc55zi2dNqiXAhCW1mB2FkYsfuIXRQM3sXAK93IbVB74R5SY5bh--zV1z0FXKc3RuzPvV20pUdgDZ9BT1N8CLR4bv7_Zm5c1RnwvkQfBc9HX0HQ0qLrD8D4uxLpyZA; __vt=ehIoA7ChUsKIE3CkABQCjdMFtf3dS_auw5cMBDN7STAbCiToZiyuHJ2YSR7kDFlKqRz37_IhsIPFD4WrNxrm6HsENGsy6k6vXgjQUGFhF0oFklEVRdAGqGpYXKkeHYQP2zEWaVW4H-iyejHNhyuKq-6UCA; TAUD=LA-1726104517893-1*RDD-1-2024_09_11*LG-169236629-2.1.F.*LD-169236630-.....; __gads=ID=85a6a8a0d9da1656:T=1718711527:RT=1726273760:S=ALNI_MYTim4zihbPMt9phRvyk0Ap4pIR8Q; __gpi=UID=00000e53b7453487:T=1718711527:RT=1726273760:S=ALNI_MYzKPLkPMlMoB2OvxgwXmwlIjAgwA; __eoi=ID=1b82a1d336e95aef:T=1718711527:RT=1726273760:S=AA-AfjY_NbUQk_TNsBknA2SqEkOR; _gcl_aw=GCL.1726273809.null; ab.storage.deviceId.6e55efa5-e689-47c3-a55b-e6d7515a6c5d=%7B%22g%22%3A%22b759bf44-b36a-f0cd-8b7d-0b1ee0f3abb1%22%2C%22c%22%3A1705328481573%2C%22l%22%3A1726273809462%7D; ab.storage.userId.6e55efa5-e689-47c3-a55b-e6d7515a6c5d=%7B%22g%22%3A%22MTA%3AD389EB225B1104D9A692107341C20D3D%22%2C%22c%22%3A1718187894267%2C%22l%22%3A1726273809463%7D; ab.storage.sessionId.6e55efa5-e689-47c3-a55b-e6d7515a6c5d=%7B%22g%22%3A%22738bf386-000b-366b-e0c6-4d26077b0fcf%22%2C%22e%22%3A1726273824901%2C%22c%22%3A1726273809462%2C%22l%22%3A1726273809901%7D; TASession=%1%V2ID.FAED50BB41FC3D26C3A1EF23F8A2045D*SQ.48*LS.Restaurant_Review*HS.recommended*ES.popularity*DS.5*SAS.popularity*FPS.oldFirst*TS.D389EB225B1104D9A692107341C20D3D*FA.1*DF.0*TRA.true*LD.50226*EAU.%28; _ga_QX0Q50ZC9P=GS1.1.1726273327.7.1.1726273814.60.0.0; idl_env_last=Sat%2C%2014%20Sep%202024%2000%3A30%3A16%20GMT; _abck=33C77055C42EFF2D92C83F677F21B7F5~-1~YAAQkHfZF+4UvuaRAQAASL7w7Qxe76dL6doOXPJUxXfqIDgM6ncTo7mmctEywPfmPhvn9JMOl3oxwPRyC0J0Oj0BJQxIF5i5ncCarHEXOgSQmHvt1Z2/58Sw1u7ZoV+JqYb1C05AcmMFTjxM8GHJEARp+AXffGrKa4JlK/HOfXLqoEuR4jBf9QskHcw/idVqbCqgUfRAfPD8RmqOFOZs5Vp2A0JfKP5Mrp9VoXftfbyhPRB+06xBxrnWho3p3t3vP9t1aHyYUvtMMV2yWr+ZzmEmqNBou8OWbcVpuzvqt5zUsH/QgYw+GPtE5FjaX0+fjmtYrPuiVbDOjr9hFycK7LmuXg0qyfWZhENbntWkLqtIcYwh4A==~-1~-1~-1; datadome=aZJ1Ws2ZJLf8Y1CZVIAIbLLCzsp4s00bcmSZLpjzDH2OtjLYOA2yx7CS_u8z~J2T69RBQZvX0JVDTCJ5E3ytS3gxX8hwWcygCP_6MtUj95wiRdp8ZJBnsBT6MMNUbmaK; OptanonConsent=isGpcEnabled=0&datestamp=Sat+Sep+14+2024+08%3A31%3A10+GMT%2B0800+(China+Standard+Time)&version=202405.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=D389EB225B1104D9A692107341C20D3D&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0004%3A1%2CC0002%3A1%2CC0003%3A1%2CV2STACK42%3A1&intType=1&geolocation=DE%3BHE&AwaitingReconsent=false",
    "origin": "https://www.tripadvisor.com",
    "priority": "u=1, i",
    
    #change
    "referer": "https://www.tripadvisor.com/Restaurant_Review-g50226-d7594515-Reviews-Fox_in_the_Snow_Cafe-Columbus_Ohio.html",
    
    "sec-ch-device-memory": "8",
    "sec-ch-ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "sec-ch-ua-arch": '"arm"',
    "sec-ch-ua-full-version-list": '"Not)A;Brand";v="99.0.0.0", "Google Chrome";v="127.0.6533.120", "Chromium";v="127.0.6533.120"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-model": '""',
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "same-origin",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

[{"data":{"cachedFilters":null,"cachedPrefs":null,"locations":[{"locationId":7594515,"name":"Fox in the Snow Cafe","placeType":"EATERY","url":"/Restaurant_Review-g50226-d7594515-Reviews-Fox_in_the_Snow_Cafe-Columbus_Ohio.html","topicCount":4,"reviewSummary":{"alertStatusCount":0,"rating":4.5,"count":309},"keywords":{"keywords":[{"keyword":"egg sandwich"},{"keyword":"pastry"},{"keyword":"cinnamon rolls"},{"keyword":"galettes"},{"keyword":"brownie"},{"keyword":"blueberries"},{"keyword":"baked goods"},{"keyword":"ham and swiss"},{"keyword":"awesome pastries"},{"keyword":"great coffee shop"},{"keyword":"italian village"},{"keyword":"the ohio state university"},{"keyword":"worth the wait"},{"keyword":"bakery"},{"keyword":"hipster"},{"keyword":"mocha"},{"keyword":"cbus"},{"keyword":"salt"}]},"reviewAggregations":{"ratingCounts":[1,1,10,54,243],"languageCounts":{"de":1,"ja":3,"en":305}},"reviewListPage":{"totalCount":305,"preferredReviewIds":[],"reviews":[{"userId":"AB807666C09FB8B32A44436CA5

### Scraper

In [230]:
for index, row in data[101:].iterrows():
    locationId = row['locationId']
    shop_url = row['url']
    
    url = "https://www.tripadvisor.com/data/graphql/ids"
    payload = [
  
    
    {
        "variables": {
            "locationId": locationId, #change
            "offset": 0,
            "limit": 15,
            "keywordVariant": "location_keywords_v2_llr_order_30_en",
            "needKeywords": True,
            "userId": "D389EB225B1104D9A692107341C20D3D",
            "filters": [
                {
                    "axis": "SORT",
                    "selections": ["mostRecent"]
                },
                {
                    "axis": "LANGUAGE",
                    "selections": ["en"]
                }
            ],
            "prefs": {
                "showMT": True,
                "sortBy": "DATE",
                "sortType": ""
            },
            "initialPrefs": {
                "showMT": True,
                "sortBy": "DATE",
                "sortType": ""
            },
            "filterCacheKey": f"locationReviewsFilters_{locationId}", #change
            "prefsCacheKey": f"locationReviewsPrefs_{locationId}" #change
        },
        "extensions": {"preRegisteredQueryId": "cbe12b7ea5ddf39c"}
    },]

    headers = {
        "accept": "*/*",
        "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
        "content-type": "application/json",
        "cookie": "TASameSite=1; TAUnique=%1%enc%3AaOaYy7VaLyHGrnrm8hGtBF8Iqr50fjk74GR2IyE67hMvwB2QVGaX4MAiq1D1yYd7Nox8JbUSTxk%3D; TASSK=enc%3AAG2kn4NbgnkJpz7gIBAvFxNTGj8jpQE7U6yP1NWumdi3AKve75DOC5%2Fcr20A%2F8MooAL%2Btex5fJWWg%2FDOxz42sjTzWCoLQpYH%2FfyUpAyeDQH5PAYVFuHXVNOcoVHc%2Flif4A%3D%3D; TART=%1%enc%3ARXtD4M8J2mDkj4k29KRxzLggMufQhBjmRz%2FQUmnFNer5xA0NlfRS0dty7jAYQSEAZQnMIP2Q3mY%3D; OptanonAlertBoxClosed=2024-09-12T01:28:04.794Z; eupubconsent-v2=CQE0XEAQE0XEAAcABBENBGFsAP_gAEPgACiQKaNV_C5ebWli8XZUIbtkaYwP55gz4kQhBhaIEewFwBOG7BgCB2EwNAV4JiACGBAAkiLBAQNlGABUCQAAAIgRiSCMYk2MgTNKJKBAiFMRO0NYCBxmmoFDWQCY5kqssxdxmBeAAAIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAAAAAAAAAAAAA_cff_9Ln_-uB_-_wU1AJMFCogBLAkICBQMIIEAAgjCAigAAAAAkBQAQAgCAJ2BAEOsJAAAAQAAgAAAABBEACAAASABCIAAACAQAAACAQABgAADAQAECAACACwAAgABANAQCAggEAwAQMQIhTAgCAAAAAAAAAAAAAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAbRlaYFo-YLntMAyQCgQAkAHQAXABsgEQAMIAnQBcgDbAIHBAAwAOgBXAEQAMIAnQCBwYAOADoALgA2QCIAGEAXIBA4QAHAB0ANkAiABhAE6ALkAgcKABgBcAMIBA4YACAMIBA4cAGAB0ARAAwgCdAIHARXIAAgDCAQOJAAwCIAGEAgcUACgA6AIgAYQBOgEDgA.f_wACHwAAAAA; OTAdditionalConsentString=1~43.46.55.61.70.83.89.93.108.117.122.124.135.143.144.147.149.159.192.196.202.211.228.230.239.259.266.286.291.311.318.320.322.323.327.367.371.385.394.397.407.415.424.430.436.445.453.486.491.494.495.522.523.540.550.559.560.568.574.576.584.587.591.737.802.803.820.821.839.864.899.904.922.931.938.979.981.985.1003.1027.1031.1040.1046.1051.1053.1067.1092.1095.1097.1099.1107.1135.1143.1149.1152.1162.1166.1186.1188.1205.1215.1226.1227.1230.1252.1268.1270.1276.1284.1290.1301.1307.1312.1345.1356.1364.1375.1403.1415.1416.1421.1423.1440.1449.1455.1495.1512.1516.1525.1540.1548.1555.1558.1570.1577.1579.1583.1584.1591.1603.1616.1638.1651.1653.1659.1667.1677.1678.1682.1697.1699.1703.1712.1716.1721.1725.1732.1745.1750.1765.1782.1786.1800.1810.1825.1827.1832.1838.1840.1842.1843.1845.1859.1866.1870.1878.1880.1889.1899.1917.1929.1942.1944.1962.1963.1964.1967.1968.1969.1978.1985.1987.2003.2008.2027.2035.2039.2047.2052.2056.2064.2068.2072.2074.2088.2090.2103.2107.2109.2115.2124.2130.2133.2135.2137.2140.2147.2150.2156.2166.2177.2183.2186.2205.2213.2216.2219.2220.2222.2225.2234.2253.2279.2282.2292.2305.2309.2312.2316.2322.2325.2328.2331.2334.2335.2336.2337.2343.2354.2357.2358.2359.2370.2376.2377.2387.2400.2403.2405.2407.2411.2414.2416.2418.2425.2440.2447.2461.2465.2468.2472.2477.2481.2484.2486.2488.2493.2498.2501.2510.2517.2526.2527.2532.2535.2542.2552.2563.2564.2567.2568.2569.2571.2572.2575.2577.2583.2584.2596.2604.2605.2608.2609.2610.2612.2614.2621.2628.2629.2633.2636.2642.2643.2645.2646.2650.2651.2652.2656.2657.2658.2660.2661.2669.2670.2677.2681.2684.2687.2690.2695.2698.2713.2714.2729.2739.2767.2768.2770.2772.2784.2787.2791.2792.2798.2801.2805.2812.2813.2816.2817.2821.2822.2827.2830.2831.2834.2838.2839.2844.2846.2849.2850.2852.2854.2860.2862.2863.2865.2867.2869.2873.2874.2875.2876.2878.2880.2881.2882.2883.2884.2886.2887.2888.2889.2891.2893.2894.2895.2897.2898.2900.2901.2908.2909.2916.2917.2918.2919.2920.2922.2923.2927.2929.2930.2931.2940.2941.2947.2949.2950.2956.2958.2961.2963.2964.2965.2966.2968.2973.2975.2979.2980.2981.2983.2985.2986.2987.2994.2995.2997.2999.3000.3002.3003.3005.3008.3009.3010.3012.3016.3017.3018.3019.3025.3028.3034.3038.3043.3048.3052.3053.3055.3058.3059.3063.3066.3068.3070.3073.3074.3075.3076.3077.3089.3090.3093.3094.3095.3097.3099.3100.3106.3109.3112.3117.3119.3126.3127.3128.3130.3135.3136.3145.3150.3151.3154.3155.3163.3167.3172.3173.3182.3183.3184.3185.3187.3188.3189.3190.3194.3196.3209.3210.3211.3214.3215.3217.3219.3222.3223.3225.3226.3227.3228.3230.3231.3234.3235.3236.3237.3238.3240.3244.3245.3250.3251.3253.3257.3260.3270.3272.3281.3288.3290.3292.3293.3296.3299.3300.3306.3307.3309.3314.3315.3316.3318.3324.3328.3330.3331.3531.3731.3831.4131.4531.4631.4731.4831.5231.6931.7235.7831.7931.8931.9731.10231.10631.10831.11031.11531.12831.13632.13731.14237.14332.15731.16831.16931.21233.23031.25731.25931.26031.26831.27731.27831.28031.28731.28831.29631.31631.32531.33631; AMZN-Token=v2FweIB2cExXMGdZTjNtOXFkczJGakFYSzh6WEVnMVJKTU15Mk1TeUJuMEg1bkFSRmIwYzhnM0FZSXdXMWt0Umd0c0lOdmxrUE9qN2ViVkdKZEJjNHBzNWdkYW9xSm9HTkNWUXNqcXNvdXFMdHozT0tiakVSaDV5ckgyK3RtanAwTUFXeGJrdgFiaXZ4GEIwOGU3Nys5SW1IdnY3MGpmZSsvdlE9Pf8=; pbjs_sharedId=383152b6-5ed0-4f0a-ac79-71e3359ac5c5; pbjs_sharedId_cst=JCzqLGQsIg%3D%3D; _gcl_au=1.1.156252779.1726104487; _ga=GA1.1.1298674465.1726104487; _lr_env_src_ats=true; idl_env_cst=JCzqLGQsIg%3D%3D; pbjs_unifiedID=%7B%22TDID_LOOKUP%22%3A%22FALSE%22%2C%22TDID_CREATED_AT%22%3A%222024-09-12T01%3A28%3A15%22%7D; pbjs_unifiedID_cst=JCzqLGQsIg%3D%3D; ServerPool=A; TATravelInfo=V2*A.2*MG.-1*HP.2*FL.3*RS.1; TAAUTHEAT=hE851IGiMfhvEIMFABQCS_DXENAPSyOjZ55LXtF4ZEAPDj2nBT5B4RZj7Yp-SLOWWlEcqiFeDHELFysFwwJ8JARxXYRrwEuo1Crik0opW3l2QdWudVx1dAOvaFhWcguRAl3VHGsDFvbLkR5C82VOZeaZ6tH1XA8wOJiPtXBcO_rY_En6_wfDlhqa-TlhK3S5nxox_PYTRdob5jdHdY_rZdYQ1L6826_I82vl; TADCID=uXfF_MTQGtPX7lH3ABQCrj-Ib21-TgWwDB4AzTFpg4A4DlugiZHaky2GOQNIVc5OymVj2fSCZAG-X8fhnG1J5pmcGate4NL9RYw; _lr_sampling_rate=100; PMC=V2*MS.31*MD.20240911*LD.20240913; TASID=FAED50BB41FC3D26C3A1EF23F8A2045D; PAC=AEIyD_yJA1AOJLXyGDWUOeXqX6cnkW1lYJ7Aisi8T9mNNsZwlkpKtqsHmzHnlttCaWFyWtG4sKWusIO9G3eOu6hsdP1Bawdi2df_igHETN9ZO8TuU1EhQCKghZ85Yku7YqsF5dy7UY7CXOKRGS6lpjao4VzHxxBE249OTlGVf9-DJZY49jNqtfIdgum1_St0cPLrk7iGnDn8JCfKq62TjMVafWkFVJjy4hhENDdqjHtw; SRT=TART_SYNC; TATrkConsent=eyJvdXQiOiJTT0NJQUxfTUVESUEiLCJpbiI6IkFEVixBTkEsRlVOQ1RJT05BTCJ9; idl_env=AjmocZ71GjxGqRYwT0JgzR62EukfsZHfUf2BFpnT3HbNw-ISWikeLdQGLWzQbD7Dwl-ghGWSTwKagZ-LDWqbn7cy07a6FhozkRZEYI1vxSZDoPfRxE82FZ9bTXCie3W9k5pCGC-AQ0_wO-mEtxrvCw-MzuvrGuYbQAlNpJM5_B-S-Ku8vA3qQnJ9mTU8xjRZENxzSIxgjmic6epXlwpEx6mC69JCQVM-xhwRkjXlswr3FQLexf_tUi5XcdPWz6iekUhqtPeapSTKVVOpuvb0w-TMn520BOD08KB3G6xMJMoL2BwYqdPD1pMNZPI0FQJ2EjK6sDwZJmSBEd52o4wfKXsZZjdf3oziRLCkzQu4OJTCSPFQGLX6XEa4kMQQP_VpO-SLpRuTDDMk2zUKFRCe9i086QahLabcdRGEvdNcO-hi-r92d2VGRUNLZLmArD9cyBSkN8XsUaLrOULQVey1in26YGyFdxe6kbilDUCrTrM-o_NLT6MnuZFey6sxxHJKWALsR-uUFZJLuRzN6WJyLR8LVW9Agknf_OnYqZgygWoYUg0ei2EP6-WSfpsY1UDMMo_tGql1yA9VpzNvs5RwPK8QI2K5vIzgpR-4DbIVjX2i0iNPUZOM_wBU9HrLBVCOrXmGSBRZanY2ez37fyfmsrYx59LHb60Oe_FkKlWaIEmpY-HFXJA6VWe-TIbeNEbB8vo28kmndkZtqthCJNCw-Y5QmRcZ1KS2xQS2XftcdW2WOV0TDhbpwE3403uOzvcHtBGJ8KO0LWf0-r24pe5XqZzXtXAbuWp4HisGeXqqrICRMDNFZlzEfPQuu4oio_bh8gyZPHj-6bdSv5ZjP93hXQluHyxDqZ-t7__-zRHtQsmmlx5dbMjwVKGH-fzbMChd5_BJPlLAk6NGe2DdSaj5FAD9gJehuS3SXoOmEQ6G-lv2CBAguhvy9Bc55zi2dNqiXAhCW1mB2FkYsfuIXRQM3sXAK93IbVB74R5SY5bh--zV1z0FXKc3RuzPvV20pUdgDZ9BT1N8CLR4bv7_Zm5c1RnwvkQfBc9HX0HQ0qLrD8D4uxLpyZA; __vt=ehIoA7ChUsKIE3CkABQCjdMFtf3dS_auw5cMBDN7STAbCiToZiyuHJ2YSR7kDFlKqRz37_IhsIPFD4WrNxrm6HsENGsy6k6vXgjQUGFhF0oFklEVRdAGqGpYXKkeHYQP2zEWaVW4H-iyejHNhyuKq-6UCA; TAUD=LA-1726104517893-1*RDD-1-2024_09_11*LG-169236629-2.1.F.*LD-169236630-.....; __gads=ID=85a6a8a0d9da1656:T=1718711527:RT=1726273760:S=ALNI_MYTim4zihbPMt9phRvyk0Ap4pIR8Q; __gpi=UID=00000e53b7453487:T=1718711527:RT=1726273760:S=ALNI_MYzKPLkPMlMoB2OvxgwXmwlIjAgwA; __eoi=ID=1b82a1d336e95aef:T=1718711527:RT=1726273760:S=AA-AfjY_NbUQk_TNsBknA2SqEkOR; _gcl_aw=GCL.1726273809.null; ab.storage.deviceId.6e55efa5-e689-47c3-a55b-e6d7515a6c5d=%7B%22g%22%3A%22b759bf44-b36a-f0cd-8b7d-0b1ee0f3abb1%22%2C%22c%22%3A1705328481573%2C%22l%22%3A1726273809462%7D; ab.storage.userId.6e55efa5-e689-47c3-a55b-e6d7515a6c5d=%7B%22g%22%3A%22MTA%3AD389EB225B1104D9A692107341C20D3D%22%2C%22c%22%3A1718187894267%2C%22l%22%3A1726273809463%7D; ab.storage.sessionId.6e55efa5-e689-47c3-a55b-e6d7515a6c5d=%7B%22g%22%3A%22738bf386-000b-366b-e0c6-4d26077b0fcf%22%2C%22e%22%3A1726273824901%2C%22c%22%3A1726273809462%2C%22l%22%3A1726273809901%7D; TASession=%1%V2ID.FAED50BB41FC3D26C3A1EF23F8A2045D*SQ.48*LS.Restaurant_Review*HS.recommended*ES.popularity*DS.5*SAS.popularity*FPS.oldFirst*TS.D389EB225B1104D9A692107341C20D3D*FA.1*DF.0*TRA.true*LD.50226*EAU.%28; _ga_QX0Q50ZC9P=GS1.1.1726273327.7.1.1726273814.60.0.0; idl_env_last=Sat%2C%2014%20Sep%202024%2000%3A30%3A16%20GMT; _abck=33C77055C42EFF2D92C83F677F21B7F5~-1~YAAQkHfZF+4UvuaRAQAASL7w7Qxe76dL6doOXPJUxXfqIDgM6ncTo7mmctEywPfmPhvn9JMOl3oxwPRyC0J0Oj0BJQxIF5i5ncCarHEXOgSQmHvt1Z2/58Sw1u7ZoV+JqYb1C05AcmMFTjxM8GHJEARp+AXffGrKa4JlK/HOfXLqoEuR4jBf9QskHcw/idVqbCqgUfRAfPD8RmqOFOZs5Vp2A0JfKP5Mrp9VoXftfbyhPRB+06xBxrnWho3p3t3vP9t1aHyYUvtMMV2yWr+ZzmEmqNBou8OWbcVpuzvqt5zUsH/QgYw+GPtE5FjaX0+fjmtYrPuiVbDOjr9hFycK7LmuXg0qyfWZhENbntWkLqtIcYwh4A==~-1~-1~-1; datadome=aZJ1Ws2ZJLf8Y1CZVIAIbLLCzsp4s00bcmSZLpjzDH2OtjLYOA2yx7CS_u8z~J2T69RBQZvX0JVDTCJ5E3ytS3gxX8hwWcygCP_6MtUj95wiRdp8ZJBnsBT6MMNUbmaK; OptanonConsent=isGpcEnabled=0&datestamp=Sat+Sep+14+2024+08%3A31%3A10+GMT%2B0800+(China+Standard+Time)&version=202405.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=D389EB225B1104D9A692107341C20D3D&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0004%3A1%2CC0002%3A1%2CC0003%3A1%2CV2STACK42%3A1&intType=1&geolocation=DE%3BHE&AwaitingReconsent=false",
        "origin": "https://www.tripadvisor.com",
        "priority": "u=1, i",

        #change
        "referer": f"https://www.tripadvisor.com{shop_url}",

        "sec-ch-device-memory": "8",
        "sec-ch-ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
        "sec-ch-ua-arch": '"arm"',
        "sec-ch-ua-full-version-list": '"Not)A;Brand";v="99.0.0.0", "Google Chrome";v="127.0.6533.120", "Chromium";v="127.0.6533.120"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-model": '""',
        "sec-ch-ua-platform": '"macOS"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "same-origin",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
    }
    
    try: 
        
        response = requests.request("POST", url, json=payload, headers=headers)
        response.raise_for_status()
        
        # Save the response JSON to a file
        file_name = str(locationId)
        file_name = f'{file_name}.json'
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'w') as json_file:
            json.dump(response.json(), json_file, indent=4)
        sleep_with_random_delay()
        
    except Exception as e:
        print(f'Exception: {e}')

Exception: HTTPSConnectionPool(host='www.tripadvisor.com', port=443): Max retries exceeded with url: /data/graphql/ids (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))
Exception: ('Connection broken: IncompleteRead(5512 bytes read, 2195 more expected)', IncompleteRead(5512 bytes read, 2195 more expected))
Exception: ('Connection broken: IncompleteRead(2756 bytes read, 2085 more expected)', IncompleteRead(2756 bytes read, 2085 more expected))
Exception: ('Connection broken: IncompleteRead(2756 bytes read, 2462 more expected)', IncompleteRead(2756 bytes read, 2462 more expected))
Exception: HTTPSConnectionPool(host='www.tripadvisor.com', port=443): Max retries exceeded with url: /data/graphql/ids (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))
Exception: HTTPSConnectionPool(host='www.tripadvisor.com', port=443): Max retries exceeded with url: 